In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.conformer as conformer
import tensorflow as tf


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
config = malaya_speech.config.transducer_featurizer_config
featurizer = malaya_speech.utils.tf_featurization.STTFeaturizer(config)

In [6]:
config = malaya_speech.config.conformer_small_encoder_config
small_model = conformer.Model(**config)

In [7]:
config = malaya_speech.config.conformer_base_encoder_config
base_model = conformer.Model(**config)

In [8]:
i = tf.placeholder(tf.float32, [None])
v = featurizer.vectorize(i)
length_v = tf.cast(tf.shape(v)[0], tf.int32)
length_v = tf.expand_dims(length_v, 0)
v = tf.expand_dims(v, 0)
v, length_v

(<tf.Tensor 'ExpandDims_3:0' shape=(1, ?, 80, 1) dtype=float32>,
 <tf.Tensor 'ExpandDims_2:0' shape=(1,) dtype=int32>)

In [9]:
small_logits = small_model(v)
small_logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<tf.Tensor 'conformer/conformer_block_15/conformer_block_15_ln/batchnorm/add_1:0' shape=(1, ?, 144) dtype=float32>

In [10]:
base_logits = base_model(v)
base_logits

<tf.Tensor 'conformer_1/conformer_block_15/conformer_block_15_ln/batchnorm/add_1:0' shape=(1, ?, 256) dtype=float32>

In [11]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [12]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [13]:
sess.run(small_logits, feed_dict = {i: y}).shape

(1, 141, 144)

In [14]:
sess.run(small_logits, feed_dict = {i: y1}).shape

(1, 88, 144)

In [15]:
sess.run(base_logits, feed_dict = {i: y}).shape

(1, 141, 256)

In [16]:
sess.run(base_logits, feed_dict = {i: y1}).shape

(1, 88, 256)